In [22]:
from glob import glob
from os.path import join, basename

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [23]:
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/sLORETA/LORETA_0,5Hz-resolution"
common_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/LORETA/LORETA_0,5Hz-resolution"
#out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/features_source_ind-bands"
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/features_source_ind-bands"
header = ['Frontal', 'Central', 'Parietal', 'Occipital', 'Temporal', 'Limbic']
freqs = np.arange(0.5, 45.5, 0.5)  # these are the lower bound frequencies of 0.5Hz bins

In [24]:
# Read IAF and TF values
iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/iaf_tf_my-boudaries.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/iaf_tf_my-boudaries.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Izmir/iaf_tf_my-boudaries.csv"
all_iaf_tf = pd.read_csv(iaf_tf_path, index_col=0)

In [25]:
all_subjects = glob(join(common_path, '*Roi.txt'))

In [26]:
def plot_individual_bands(bands: dict[str, tuple[float, float]], tf: float, iaf: float, subject_code: str):
    # Make a horizonal plot with the bands on the x-axis
    fig, ax = plt.subplots(1, 1, figsize=(20, 3))
    for i, (band, (inf, sup)) in enumerate(bands.items()):
        # coarse line
        plt.hlines(i, inf, sup, label=band, color='black', lw=4)
    # plot vertically dashed in red the TF and IAF
    plt.vlines(tf, -1, len(bands), color='red', linestyles='dashed', lw=2, label='TF')
    plt.vlines(iaf, -1, len(bands), color='blue', linestyles='dashed', lw=2, label='IAF')
    ax.set_yticks(range(len(bands)))
    ax.set_yticklabels(list(bands.keys()))
    ax.set_xlabel("Frequency (Hz)")
    # make all x-ticks from 'bands' and add TF and IAF
    xticks = list(bands.values())
    xticks.append((tf, tf))
    xticks.append((iaf, iaf))
    ax.set_xticks([x for x, _ in xticks])
    ax.set_title("Individual bands of subject " + subject_code)
    #plt.show()
    plt.savefig(join(out_common_path, f"{subject_code}_bands.png"), dpi=300, bbox_inches='tight')
    plt.close()

In [27]:
def average_band(_df, inf, sup, label, verbose=False):
    if verbose:
        print("Inf:", inf, "Sup:", sup)
        
    # Discretize input
    # If inf or sup are not .0 or .5, round to the nearest .0 or .5
    inf = round(inf*2)/2
    sup = round(sup*2)/2
    
    # Average by column
    res = _df.loc[inf: sup-0.5].mean(axis=0)
    
    # Print the index of all rows that were averaged
    bins = _df.loc[inf: sup-0.5].index.tolist()
    n_bins = len(bins)
    if verbose:
        print(label, "| Averaging", bins)
        
    return res, pd.Series({l: n_bins for l in header})

In [28]:
def flatten_df(_df):
    # E.g. Delta_Frontal, Delta_Central, Delta_Parietal, Delta_Occipital, Delta_Temporal, Delta_Limbic, ...
    _res = {}
    for band, band_df in _df.items():
        for region, value in band_df.items():
            _res[f"{band}_{region}"] = value
    _res = pd.DataFrame(_res, index=[0])
    return _res

In [29]:
res = pd.DataFrame(dtype=float)
res_number_bins = pd.DataFrame(dtype=int)
skipped_subjects = 0
low_tf_subjects = 0
for filepath in all_subjects:
    #subject_code = basename(filepath).split('-')[0].split(' ')[1]
    subject_code = basename(filepath).split('-')[0]
    #subject_code = basename(filepath).split('-')[0].split('PARTICIPANT')[1]
    print("Subject code:", subject_code)
    subject_out_filepath = join(out_common_path, f"{subject_code}.csv")
    
    # Read .txt file to Dataframe
    df = pd.read_csv(filepath, sep='  ', header=None, engine='python')
    df.columns = header
    df.index = freqs
    assert df.shape == (90, 6)
    
    # Get IAF and TF of this subject
    #iaf, tf = all_iaf_tf.loc[int(subject_code)]
    iaf, tf = all_iaf_tf.loc[subject_code]
    print("IAF:", iaf, "TF:", tf)
    iaf, tf = round(iaf, 1), round(tf, 1)  # round to 1 decimal place
    
    # Extract/average values per individual band
    band_features = {}
    band_bins = {}
    
    if tf >= iaf:
        print("TF is greater than or equal to IAF. Skipping...")
        skipped_subjects += 1
        continue
     
    if tf < 4.5:
        print("TF is less than 4Hz, hence no Delta band")#, Skipping...")
        low_tf_subjects += 1
        #continue
    
    # Delta
    delta_inf = tf - 4 if tf - 4 >= 2 else 2
    delta_sup = tf-2 if tf-2 > delta_inf else 3
    band_features['Delta'], band_bins['Delta'] = average_band(df, delta_inf, delta_sup, 'Delta', verbose=True)
    # Alpha 1
    alpha1_sup = tf + ((iaf - tf) / 2)
    alpha1_inf = tf if alpha1_sup - tf >= 0.5 else alpha1_sup - 0.5
    if alpha1_inf < 6:
        alpha1_inf = 6 
        alpha1_sup = 6.5
    band_features['Alpha1'], band_bins['Alpha1'] = average_band(df, alpha1_inf, alpha1_sup, 'Alpha1', verbose=True)
    # Theta
    theta_inf = tf-2 if tf-2 >= delta_sup else delta_sup
    theta_sup = tf if tf - theta_inf >= 2 else alpha1_inf
    if theta_sup > alpha1_inf:
        theta_sup = alpha1_inf
    band_features['Theta'], band_bins['Theta'] = average_band(df, theta_inf, theta_sup, 'Theta', verbose=True)
    # Alpha 2
    alpha2_inf = alpha1_sup
    alpha2_sup = iaf if iaf - alpha2_inf >= 0.5 else alpha2_inf + 1.0
    band_features['Alpha2'], band_bins['Alpha2'] = average_band(df, alpha2_inf, alpha2_sup, 'Alpha2', verbose=True)
    # Alpha 3
    alpha3_inf = iaf if iaf >= alpha2_sup else alpha2_sup
    alpha3_sup = iaf + 2 if iaf + 2 <= 13 else 13
    band_features['Alpha3'], band_bins['Alpha3'] = average_band(df, alpha3_inf, alpha3_sup, 'Alpha3')
    # Beta 1
    beta1_inf = 14
    beta1_sup = 20
    band_features['Beta1'], band_bins['Beta1'] = average_band(df, beta1_inf, beta1_sup, 'Beta1')
    # Beta 2
    beta2_inf = 20
    beta2_sup = 30
    band_features['Beta2'], band_bins['Beta2'] = average_band(df, beta2_inf, beta2_sup, 'Beta2')
    # Gamma
    gamma_inf = 30
    gamma_sup = 40
    band_features['Gamma'], band_bins['Gamma'] = average_band(df, gamma_inf, gamma_sup, 'Gamma')
    
    # Plot individual bands
    plot_individual_bands({'Delta': (delta_inf, delta_sup), 'Theta': (theta_inf, theta_sup), 'Alpha1': (alpha1_inf, alpha1_sup), 
                           'Alpha2': (alpha2_inf, alpha2_sup), 'Alpha3': (alpha3_inf, alpha3_sup), 'Beta1': (beta1_inf, beta1_sup)},
                          tf, iaf, subject_code)
    
    # Flatten band_features
    subject_res = flatten_df(band_features)
    subject_res.index = [subject_code]
    
    subject_res_bins = flatten_df(band_bins)
    subject_res_bins.index = [subject_code] 
    
    # Add to res using concat
    res = pd.concat([res, subject_res], axis=0)
    res_number_bins = pd.concat([res_number_bins, subject_res_bins], axis=0)

Subject code: alzhay1
IAF: 9.0 TF: 5.5
Inf: 2 Sup: 3.5
Delta | Averaging [2.0, 2.5, 3.0]
Inf: 6 Sup: 6.5
Alpha1 | Averaging [6.0]
Inf: 3.5 Sup: 5.5
Theta | Averaging [3.5, 4.0, 4.5, 5.0]
Inf: 6.5 Sup: 9.0
Alpha2 | Averaging [6.5, 7.0, 7.5, 8.0, 8.5]
Subject code: alzhpeh1
IAF: 8.0 TF: 4.0
TF is less than 4Hz, hence no Delta band
Inf: 2 Sup: 3
Delta | Averaging [2.0, 2.5]
Inf: 6 Sup: 6.5
Alpha1 | Averaging [6.0]
Inf: 3 Sup: 6
Theta | Averaging [3.0, 3.5, 4.0, 4.5, 5.0, 5.5]
Inf: 6.5 Sup: 8.0
Alpha2 | Averaging [6.5, 7.0, 7.5]
Subject code: alzmsap1
IAF: 8.0 TF: 7.5
Inf: 3.5 Sup: 5.5
Delta | Averaging [3.5, 4.0, 4.5, 5.0]
Inf: 7.25 Sup: 7.75
Alpha1 | Averaging [7.0, 7.5]
Inf: 5.5 Sup: 7.25
Theta | Averaging [5.5, 6.0, 6.5]
Inf: 7.75 Sup: 8.75
Alpha2 | Averaging [8.0, 8.5]
Subject code: d1kyil1
IAF: 9.5 TF: 7.5
Inf: 3.5 Sup: 5.5
Delta | Averaging [3.5, 4.0, 4.5, 5.0]
Inf: 7.5 Sup: 8.5
Alpha1 | Averaging [7.5, 8.0]
Inf: 5.5 Sup: 7.5
Theta | Averaging [5.5, 6.0, 6.5, 7.0]
Inf: 8.5 Sup: 9.5


In [30]:
print("Skipped subjects:", skipped_subjects)
print("Low TF subjects:", low_tf_subjects)

Skipped subjects: 0
Low TF subjects: 17


In [31]:
res

,Delta_Frontal,Delta_Central,Delta_Parietal,Delta_Occipital,Delta_Temporal,Delta_Limbic,Alpha1_Frontal,Alpha1_Central,Alpha1_Parietal,Alpha1_Occipital,...,Beta2_Parietal,Beta2_Occipital,Beta2_Temporal,Beta2_Limbic,Gamma_Frontal,Gamma_Central,Gamma_Parietal,Gamma_Occipital,Gamma_Temporal,Gamma_Limbic
alzhay1,2.308504,2.297174,2.232657,2.661051,3.369434,1.638357,1.715127,1.607953,1.284550,1.632907,...,0.573062,0.718002,0.686041,0.334631,0.285755,0.234828,0.286601,0.605088,0.507792,0.174364
alzhpeh1,3.098892,3.071003,4.012073,2.375318,3.060319,1.661857,4.104218,4.011325,4.324359,2.733344,...,0.303614,0.163487,0.256007,0.169680,0.186262,0.163689,0.150894,0.104935,0.162845,0.079347
alzmsap1,2.666047,3.865545,3.625338,1.171270,2.171202,1.414644,1.015967,1.700584,1.617200,0.494176,...,0.472339,0.134389,0.204927,0.155014,0.221978,0.353993,0.281030,0.088154,0.174100,0.126660
d1kyil1,1.898084,2.403434,2.696341,1.775227,2.140795,1.318017,1.203933,1.482436,1.824735,1.421160,...,0.512586,0.385411,0.355376,0.204375,0.202817,0.234535,0.268390,0.261193,0.239247,0.109506
eoabob1,0.549071,1.433651,1.187403,0.905909,0.677006,0.526163,0.358050,1.005565,0.810676,0.713222,...,0.372614,0.224167,0.233451,0.212556,0.088191,0.181211,0.124451,0.083945,0.092601,0.063414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pspam1,2.770660,1.029284,1.178939,1.836590,2.710647,1.196248,0.789880,0.328347,0.632466,1.419869,...,0.305978,0.337835,0.400985,0.202643,0.141921,0.096378,0.085664,0.086207,0.114226,0.061479
pssen1,1.317364,3.734599,3.230311,1.881036,1.631997,1.010162,0.680026,1.628276,1.773899,1.259264,...,0.605467,0.406876,0.430233,0.223215,0.131080,0.255611,0.276991,0.254032,0.254304,0.111000
pssog1,2.961500,5.259034,3.996294,2.333989,1.511971,1.984684,2.286720,3.246640,1.759820,1.076476,...,0.367283,0.221740,0.174833,0.218261,0.180867,0.284205,0.203255,0.146231,0.129747,0.106277
psyil1,1.495460,1.455166,1.496903,1.340814,3.262905,0.971026,0.814404,0.658180,0.790039,0.854817,...,0.308143,0.330845,0.565508,0.198239,0.288837,0.199848,0.178999,0.192060,0.361015,0.130120


In [32]:
res_number_bins

,Delta_Frontal,Delta_Central,Delta_Parietal,Delta_Occipital,Delta_Temporal,Delta_Limbic,Alpha1_Frontal,Alpha1_Central,Alpha1_Parietal,Alpha1_Occipital,...,Beta2_Parietal,Beta2_Occipital,Beta2_Temporal,Beta2_Limbic,Gamma_Frontal,Gamma_Central,Gamma_Parietal,Gamma_Occipital,Gamma_Temporal,Gamma_Limbic
alzhay1,3,3,3,3,3,3,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
alzhpeh1,2,2,2,2,2,2,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
alzmsap1,4,4,4,4,4,4,2,2,2,2,...,20,20,20,20,20,20,20,20,20,20
d1kyil1,4,4,4,4,4,4,2,2,2,2,...,20,20,20,20,20,20,20,20,20,20
eoabob1,3,3,3,3,3,3,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pspam1,1,1,1,1,1,1,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
pssen1,1,1,1,1,1,1,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
pssog1,3,3,3,3,3,3,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
psyil1,4,4,4,4,4,4,3,3,3,3,...,20,20,20,20,20,20,20,20,20,20


In [33]:
# Save res
res.to_csv(join(out_common_path, "all_features.csv"))